In [1]:
import matplotlib
matplotlib.use('Agg')

import numpy as np

import skimage
import skimage.io
import skimage.transform
import skimage.exposure 

from quilt.data.jared import landuse_austin_tx

%matplotlib inline

In [2]:
def findimg(attop,root,layer) :
    atdir = getattr(attop,root)
    atimg = getattr(atdir,root+layer)
    return atimg

In [3]:
def prepimage(attop,root): # prepare the image stack
    
    tmpimg = findimg(attop,root,'_B4')
    r=skimage.io.imread(tmpimg(), as_grey=True) # load red
    tmpimg = findimg(attop,root,'_B3')
    g=skimage.io.imread(tmpimg(), as_grey=True) # load green
    tmpimg = findimg(attop,root,'_B2')
    b=skimage.io.imread(tmpimg(), as_grey=True) # load blue
    tmpimg = findimg(attop,root,'_B5')
    ir=skimage.io.imread(tmpimg(), as_grey=True)# load band 5 - near infrared
    
    ndvi = np.true_divide((ir - r), (ir + r)) # ndvi 
    rgb= np.dstack((r,g,b))                   # combine into ordered stack

    tmpimg = findimg(attop,root,'_B8') # load hi-res image
    pan=skimage.io.imread(tmpimg(), as_grey=True)/65535 # load chromatic, normalize to between 0 and 1

    rgb_big=skimage.transform.resize(rgb, output_shape=(pan.shape[0],pan.shape[1],3), order=3, mode='constant', cval=0.0) # resize the rgb composite to match the chromatic
    ndvi_big=skimage.transform.resize(ndvi, output_shape=(pan.shape[0],pan.shape[1]), order=3, mode='constant', cval=0.0) # resize ndvi to match the rgb

    hsv = skimage.color.rgb2hsv(rgb_big) #Convert the rgb to the HSV colorspace
    hsv[...,2]=pan                       #Pan Sharpen
    rgb_pan = skimage.color.hsv2rgb(hsv) #Convert back to rgb space
    
    return rgb_pan, ndvi_big 

In [4]:
attop = landuse_austin_tx.images.cropped               # use precropped images 
rgb, ndvi = prepimage(attop,'LC80270392014022LGN00')   # test image

In [5]:
from scipy import ndimage as ndi
rgb_dog = rgb - ndi.gaussian_filter(rgb, 15) # difference of gaussians 'edge finder'

In [6]:
rgb_light  = skimage.exposure.rescale_intensity(rgb,in_range=(0,0.5))  #lighten image for viewer
ndvi_light = skimage.exposure.rescale_intensity(ndvi,in_range=(0,0.5)) #lighten image for viewer

from PIL import Image
img = Image.fromarray(np.uint8(rgb_light*255.99))
img.save('../output/LC80270392014022LGN00_rgb.tif')

img = Image.fromarray(np.uint8(ndvi_light*255.99))
img.save('../output/LC80270392014022LGN00_ndvi.tif')

img = Image.fromarray(np.uint8(rgb_dog*255.99))
img.save('../output/LC80270392014022LGN00_dog.tif')